In [1]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
from formatting import comma_del, remove_zeros

In [2]:
# Iterates through all csv files in 'County_Data_Clean' and produces a dictionary containing county abbreviations and 
# file names. 
files = os.listdir("County_Data_Clean")
counties = [x.split("_")[0] for x in files]
file_dict = {counties[i]:files[i] for i in range(len(files))}

# Outputs a CSV of county abbreviations
pd.DataFrame(counties).to_csv("Counties.csv")

In [3]:
# Imports Voter Registration Data For Precincts.
precinct_df = pd.read_csv("2018gen_precinct.csv")

# Drops blank row.
precinct_df = precinct_df.drop(5881)

In [4]:
# Creates Dictionary of Data Frames.
# Example: {..., "SEM":<data frame with clean Seminole County data>, ...}
county_df_dict = {}
for county in counties:
    path = os.path.join("County_Data_Clean",file_dict[county])
    county_df_dict[county] = pd.read_csv(path,index_col=0)

In [5]:
# Removes Leading Zeros from Precinct Data.
precinct_df['Precinct Number'].map(remove_zeros)
precinct_df

,County Code,Precinct Number,Republican Party of Florida,Florida Democratic Party,Other,Total
0,ALA,1,584,538,287,"1,409"
1,ALA,2,739,951,379,"2,069"
2,ALA,3,"1,626","1,520",961,"4,107"
3,ALA,4,"1,406","1,040",671,"3,117"
4,ALA,5,733,"1,552",841,"3,126"
...,...,...,...,...,...,...
5876,WAS,5,729,546,247,"1,522"
5877,WAS,6,221,318,85,624
5878,WAS,7,810,507,226,"1,543"
5879,WAS,8,764,696,175,"1,635"


In [6]:
# Reformats precinct numbers for merge
for county in county_df_dict:
    county_df_dict[county] = county_df_dict[county].rename(columns = {"Unique PCT Identifier":"Precinct Number"})
    county_df_dict[county]['Precinct Number'] = county_df_dict[county]['Precinct Number'].map(str)
    county_df_dict[county]['Precinct Number'] = county_df_dict[county]['Precinct Number'].map(remove_zeros)  

In [7]:
# Separates precinct data into a dictionary of data frames.
precinct_dict={}
for county in counties:
    precinct_dict[county]=precinct_df[precinct_df['County Code']==county]

In [8]:
# Merges precinct data into elections data for each county.
for county in counties:
    county_df_dict[county] = pd.merge(county_df_dict[county],precinct_dict[county],
                                      on=["Precinct Number","County Code"],
                                      how="inner")
    
    county_df_dict[county] = county_df_dict[county].rename(columns = {"Republican Party of Florida":"Registered Republicans",
                                     "Florida Democratic Party":"Registered Democrats",
                                     "Other":"Registered Other"})
    
    county_df_dict[county]['Total'] = county_df_dict[county]['Total'].map(comma_del)

In [9]:
county_precinct_gb_dict = {}

for county in counties:
    county_precinct_gb_dict[county] = county_df_dict[county].groupby('Precinct Number')


In [17]:
for county in counties:
    for precinct in county_precinct_gb_dict[county].groups.keys():
        df = county_precinct_gb_dict[county].get_group(precinct)
        turnout_vals = df['Total Registered'].map(int)
        precinct_vals = df['Total'].map(int)
        discrepancy = abs(max(turnout_vals) - max(precinct_vals))
        tolerance = .005*( max(turnout_vals) + max(precinct_vals) )
        if max(precinct_vals)==0:
            print(f"{county},{precinct}: p=0")
        if max(turnout_vals)==0:
            print(f"{county},{precinct}: t=0")
        if discrepancy > tolerance:
            print(f"{county} , {precinct} : {max(turnout_vals)} , {max(precinct_vals)}")
        #if sum((df['Total Registered'].map(int) - df['Vote Total'].map(int)).map(abs)) != 0:
            #print(county,precinct)

BRO , L015 : 9560 , 956
CAL , 101 : 1176 , 564
CAL , 401 : 781 , 244
CAL , 501 : 882 , 547
DAD , 100 : 3951 , 7479
DAD , 107 : 938 , 949
DAD , 11 : 1809 , 1955
DAD , 124 : 2083 , 3289
DAD , 161 : 1696 , 1716
DAD , 167 : 448 , 507
DAD , 172 : 182 , 185
DAD , 182 : 764 , 1010
DAD , 196 : 2216 , 2847
DAD , 202 : 2057 , 2190
DAD , 203 : 1211 , 2908
DAD , 205 : 1997 , 2040
DAD , 206 : 1252 , 2365
DAD , 207 : 2937 , 3192
DAD , 213 : 1579 , 2915
DAD , 215 : 1742 , 2751
DAD , 221 : 1259 , 2447
DAD , 222 : 1983 , 2564
DAD , 233 : 1777 , 2630
DAD , 248 : 1833 , 2887
DAD , 250 : 1653 , 2565
DAD , 251 : 2381 , 2690
DAD , 256 : 1344 , 2238
DAD , 258 : 718 , 1132
DAD , 261 : 1657 , 2321
DAD , 266 : 94 , 97
DAD , 269 : 1008 , 1600
DAD , 281 : 220 , 355
DAD , 295 : 962 , 3196
DAD , 300 : 2883 , 4017
DAD , 304 : 2857 , 3393
DAD , 341 : 25 , 26
DAD , 403 : 929 , 1362
DAD , 411 : 1084 , 2154
DAD , 413 : 2861 , 3231
DAD , 424 : 2303 , 2751
DAD , 430 : 1607 , 2426
DAD , 431 : 1185 , 1541
DAD , 434 : 1085 ,